In [ ]:
import spacy
import seaborn as sns
import numpy as np
from spacy import displacy
from collections import Counter
from collections import defaultdict
from matplotlib import pyplot as plt

sns.set_style("darkgrid")
nlp = spacy.load('en_core_web_sm')


# This function depends on a vocabulary as created by 
#def create_integer_vocabulary(word_freqs, max_voc_size):
#    """ 
#    Create vocabulary where common words are matched to integers. 
#    """
#    
#    word_list = []
#
#    if len(word_freqs.most_common()) > max_voc_size:
#        vocab = word_freqs.most_common()[0:max_voc_size]
#
#    else:
#        vocab = word_freqs
#
#    for i in range(len(vocab)):
#        word_list.append(vocab[i][0])
#
#    # Get pairs of elements    
#    tmp = zip(word_list, range(1,max_voc_size+1))
#    # Make pairs into a dictionary
#    vocab = dict(tmp)
#
#    # Create default dictionary - returns 0 if an undefined key is called
#    vocab2 = defaultdict(int)
#    vocab2.update(vocab)
#
#    # Double check that it returns 0
#    # print(vocab2["terehgdjhshrersg"])
#    
#    return vocab2

def create_batches(batch_size,vocabulary,filename,ENCODING):
    counter=0 # will end up being the number of lines in the document
    len_lines = [] # will contain maximum length of a line in each batch
    tmp_lines = []
    
    with open(filename, encoding=ENCODING) as f:
        for line in f:
            counter+=1
            tokens = line.lower().split()
            tmp_lines.append(len(tokens))

            if (counter % batch_size == 0):
                len_lines.append(max(tmp_lines))
                tmp_lines = []
        #This takes care of the last batch if number of lines is not an exact multiple of batch_size
        if (counter % batch_size != 0): 
            len_lines.append(max(tmp_lines)) # if at end of the file
    
    with open(filename, encoding=ENCODING) as f:
        batches=[]
        batch_counter=0
        line_counter=0

        for line in f:
            #This creates a temporary array each time we start a new batch
            if line_counter % batch_size == 0:
                tmp_array=np.zeros(shape=(batch_size,len_lines[batch_counter])) #fill this temporary array


            tokens = line.lower().split()
            line_as_int = list(map(vocabulary.get, tokens))
            line_as_int = [-1 if x is None else x for x in line_as_int] # set None values to -1


            tmp_array[line_counter % batch_size,0:(len(line_as_int))]=line_as_int

            line_counter+=1 #when we done
            if line_counter % batch_size ==0:
                batches.append(tmp_array)
                batch_counter+=1

        # again this takes care of the final batch if number of lines is not multiple of batch_size
        if line_counter % batch_size != 0:
            tmp_array=tmp_array[0:(line_counter % batch_size),:]
            batches.append(tmp_array)
        
    return (counter,batches)


# Here's an example run:

In [ ]:
max_voc_size = 1000
book_freqs = count_word_frequencies("a1_data/books.txt", 'ISO-8859-1')
vocab_books = create_integer_vocabulary(book_freqs, max_voc_size)
num_lines,batches=create_batches(batch_size=101,vocabulary=vocab_books,filename='a1_data/books.txt',ENCODING='ISO-8859-1')